**disclaimer:** This notebook was used for manipulation and preprocessing of our data as an interactive script. Its execution might encounter Errors by the fact that files and folders might have changed path or have been renamed.

# merging csvs
This notebook was built to merge waze data from flood and general alerts, so that we could use them as input to visualize maps of kepler.gl.

In [2]:
import os
import pandas as pd
import numpy as np
import csv
#import re
#import pytz
#from shapely.geometry import Point

In [3]:
import sys
from pathlib import Path

current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

city = 'Rio de Janeiro'

RAW_PATH = current_path.parent / 'data' / 'raw' / city
OUTPUT_PATH = current_path.parent / 'data' / 'output' / city

datapath = "../data/"
outputs = "../outputs/"

# links
* https://darksky.net/dev/docs
* https://github.com/bitpixdigital/forecastiopy3

# Exploring metadata

In [4]:
df_rj1 = pd.read_csv(RAW_PATH/'Rio-Worst-Day-Flood.csv')
df_rj2 = pd.read_csv(RAW_PATH/'Rio-Worst-Day-irreg.csv')

In [79]:
df_rj1.head(5)

,pub_utc_date,points
0,2019-04-09 08:20:00.000,POINT (-43.55413 -22.871565)
1,2019-04-09 08:20:00.000,POINT (-43.508555 -22.863237)
2,2019-04-09 08:20:00.000,POINT (-43.367349 -22.97356)
3,2019-04-09 08:20:00.000,POINT (-43.357138 -22.966433)
4,2019-04-09 08:20:00.000,POINT (-43.359448 -22.924806)


In [78]:
df_rj2.head(5)

,pub_utc_date,lines
0,2019-04-09 08:12:00.000,"LINESTRING (-43.216866 -22.963617, -43.217381 ..."
1,2019-04-09 08:11:00.000,"LINESTRING (-43.216866 -22.963617, -43.217381 ..."
2,2019-04-09 12:01:00.000,"LINESTRING (-43.366809 -22.838573, -43.380171 ..."
3,2019-04-09 12:01:00.000,"LINESTRING (-43.185941 -22.943584, -43.185947 ..."
4,2019-04-09 12:01:00.000,"LINESTRING (-43.397092 -22.898731, -43.397183 ..."


In [52]:
df_rj1 = df_rj1[['pub_utc_date', 'longitude', 'latitude']]
df_rj1['points'] = df_rj1.apply(lambda row: Point(row['longitude'], row['latitude']), 1)
df_rj1 = df_rj1[['pub_utc_date', 'points']]

In [53]:
df_rj2 = df_rj2[['pub_utc_date', 'geometry']].rename(columns={'geometry':'lines'})

In [56]:
df = pd.concat([df_rj1, df_rj2], ignore_index=True, sort=False)

In [60]:
df['pub_utc_date'] = df.pub_utc_date.apply(pd.to_datetime).dt.tz_localize('UTC')

In [62]:
df.head()

,pub_utc_date,points,lines
0,2019-04-09 08:20:00+00:00,POINT (-43.55413 -22.871565),NaN
1,2019-04-09 08:20:00+00:00,POINT (-43.508555 -22.863237),NaN
2,2019-04-09 08:20:00+00:00,POINT (-43.367349 -22.97356),NaN
3,2019-04-09 08:20:00+00:00,POINT (-43.357138 -22.966433),NaN
4,2019-04-09 08:20:00+00:00,POINT (-43.359448 -22.924806),NaN


In [72]:
df['pub_utc_date'] = df.pub_utc_date.dt.tz_convert('America/Sao_Paulo').apply(lambda x: x.replace(tzinfo=None))

In [74]:
df = df.rename(columns={'pub_utc_date':'pub_local_date'})

In [76]:
file_name = os.path.join(datapath,'Rio-Worst-Day-all.csv')

In [77]:
df.to_csv(file_name, index=False)